### 휴먼지능정보공학과 201910787 김영서

- 문제 1: 오픈API를 통해 데이터를 5천 ~ 1만 건 정도 수집해서 파일로 저장한다 (초과해도 좋다).<br>
열린데이터, 공공데이터 또는 (할 수 있다면) 트위터와 같은 SNS, 댓글 등 적합한 소스를 자신이 선택한다.<br>
과정의 후반으로 가서, 이 데이터를 분석하는 과제가 주어지게 된다.<br>
프로그램에서 몇 건을 읽었는지와 마지막 줄을 화면 출력한다.<br>
!dir 명령어로 파일이 존재하는지 출력한다. 1만 건 정도 저장되었으니 파일의 크기가 적당한지 가늠해보자



- 문제 2: 노트북에서 pyspark 실행<br>
주피터 노트북을 실행한 후, 셀에서 아래 명령어를 실행하여 spark를 생성하고, 그 버전을 출력하세요. <br>

[In] spark.version <br>
[Out] '3.x.x'<br>
그리고 다음과 같이 버전을 출력한다.<br>
pyspark는 jdk가 설치되어 있어야 한다.<br>
!java --version   #spark v2.x는 java 8이 적당.<br>
!javac --version  #java 버전과 동일한지 확인한다.<br>
!python --version  <br>
!python3 --version # python 2, 3 멀티 버전이 설치된 경우에만 출력<br>
import sys<br>
print(sys.version) # 자신이 사용하는 python 버전과 동일한지 확인한다.<br>



- 문제 3: 다음 파일을 읽어서 RDD를 생성하고, 5줄을 화면출력하세요.<br>
결과가 깨져보인다면, 그 이유를 간단히 적으세요.<br>

1) 경기도 의정부시 인구현황 (파일명: ```경기도 의정부시_인구현황_20200904```)<br>
https://www.data.go.kr/data/15009613/fileData.do


2) 제주특별자치도 서귀포시 내 연도별 65세이상 인구수 및 고령화비율, 노령화지수 현황 (파일명: ```제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623```)<br>
https://www.data.go.kr/data/15051545/fileData.do

### 문제1 답

* 서울시 개인서비스 요금 정보<br>
http://data.seoul.go.kr/dataList/OA-1169/S/1/datasetView.do

### 인증키

In [99]:
%%writefile src/key.properties
dataseoul=4e6b7476757a617133384842775573
gokr=8Bx4C1%2Bbogiu3i0M5k7aqOf%2BTA5yCWK%2BFDI7nTSJ0oAhDj1NC5CtbI1TMgFmnpuZxJ9LBoCScqBhuw5sul2tmQ%3D%3D

Overwriting src/key.properties


In [100]:
import os
import src.mylib

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=src.mylib.getKey(keyPath)
KEY=str(key['dataseoul'])

### 요청인자

In [101]:
# api
TYPE='json'
SERVICE='IndividualServiceChargeService'
START_INDEX=str(1)
END_INDEX=str(10)

### url

In [102]:
params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX])
print(params)

4e6b7476757a617133384842775573/json/IndividualServiceChargeService/1/10


In [103]:
import urllib
_url='http://openAPI.seoul.go.kr:8088/'
url=urllib.parse.urljoin(_url,params)
print(url) # backslashes removed

http://openAPI.seoul.go.kr:8088/4e6b7476757a617133384842775573/json/IndividualServiceChargeService/1/10


### url 호출

In [104]:
import requests

r=requests.get(url)

In [105]:
charge=r.json()

In [106]:
print(charge)

{'IndividualServiceChargeService': {'list_total_count': 7835, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'BSSH_NM': '핑클세탁소', 'BSSH_SEQ_NO': 1418890707533.0, 'INDUTY_DESC': '세탁업', 'TLPHON_NO_CN': '02-888-8380', 'AR': '', 'LEGALDONG_NM': '봉천동', 'ADRES_CN2': '서울특별시 관악구 관악로37길 15 (봉천동)', 'PRDLST_CODE_SE': '1418622761392', 'PRDLST_DESC': '양복 세탁료', 'PC': 8000.0, 'CHCK_MD': '2021-09-15 00:00:00.0', 'GU_NM': '관악구'}, {'BSSH_NM': '전주식당', 'BSSH_SEQ_NO': 1501655118455.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-873-0157', 'AR': '', 'LEGALDONG_NM': '봉천동', 'ADRES_CN2': '서울특별시 관악구 관악로14길 105 (봉천동)', 'PRDLST_CODE_SE': '1418622761373', 'PRDLST_DESC': '삼겹살', 'PC': 13000.0, 'CHCK_MD': '2021-09-15 00:00:00.0', 'GU_NM': '관악구'}, {'BSSH_NM': '전주식당', 'BSSH_SEQ_NO': 1501655118455.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-873-0157', 'AR': '', 'LEGALDONG_NM': '봉천동', 'ADRES_CN2': '서울특별시 관악구 관악로14길 105 (봉천동)', 'PRDLST_CODE_SE': '1418622761372', 'PRDLST_DESC': '된장찌개 백반', 'PC': 7000.0, 'CHCK

In [168]:
%%writefile src/ds_open_IndividualServiceChargeService_json.py
#!/usr/bin/env python
#coding: utf-8
import sys
import os
import requests
import urllib
import mylib # NO! src.mylib
    
def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY=str(key['dataseoul'])
    # api
    TYPE='json'
    SERVICE='IndividualServiceChargeService'
    START_INDEX=str(1)
    END_INDEX=str(10)
    startIndex=1
    endIndex=10
    list_total_count=0
    
    while True:
        #sys.stdout = open('src/ds_open_IndividualServiceChargeService_xml.txt', 'w')
        START_INDEX=str(startIndex)
        END_INDEX=str(endIndex)
        params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX])
        _url='http://openAPI.seoul.go.kr:8088' 
        url="/".join([_url,params])
        r=requests.get(url)
        charge=r.json()
        if(startIndex==1):
            list_total_count=charge['IndividualServiceChargeService']['list_total_count']
            print("---- 가져온 데이터 건 수:", list_total_count)
        startIndex+=10
        endIndex+=10
        if(endIndex > list_total_count):
            for e in charge['IndividualServiceChargeService']['row']:
                print (charge['IndividualServiceChargeService']['row'])
            print("----- Ending endIndex=",endIndex)
            #sys.stdout.close()
            break

if __name__ == "__main__":
    doIt()


Overwriting src/ds_open_IndividualServiceChargeService_json.py


In [169]:
!python3 src/ds_open_IndividualServiceChargeService_json.py

---- 가져온 데이터 건 수: 7835
[{'BSSH_NM': '삼지', 'BSSH_SEQ_NO': 1418890661033.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-763-8264', 'AR': '', 'LEGALDONG_NM': '재동', 'ADRES_CN2': '서울특별시 종로구 북촌로2길 5-7 (재동)', 'PRDLST_CODE_SE': '1418622761384', 'PRDLST_DESC': '김치찌개 백반', 'PC': 7000.0, 'CHCK_MD': '2021-08-18 00:00:00.0', 'GU_NM': '종로구'}, {'BSSH_NM': '삼지', 'BSSH_SEQ_NO': 1418890661033.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-763-8264', 'AR': '', 'LEGALDONG_NM': '재동', 'ADRES_CN2': '서울특별시 종로구 북촌로2길 5-7 (재동)', 'PRDLST_CODE_SE': '1418622761373', 'PRDLST_DESC': '삼겹살', 'PC': 14000.0, 'CHCK_MD': '2021-08-18 00:00:00.0', 'GU_NM': '종로구'}, {'BSSH_NM': '삼지', 'BSSH_SEQ_NO': 1418890661033.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-763-8264', 'AR': '', 'LEGALDONG_NM': '재동', 'ADRES_CN2': '서울특별시 종로구 북촌로2길 5-7 (재동)', 'PRDLST_CODE_SE': '1418622761372', 'PRDLST_DESC': '된장찌개 백반', 'PC': 7000.0, 'CHCK_MD': '2021-08-18 00:00:00.0', 'GU_NM': '종로구'}, {'BSSH_NM': '김밥짱', 'BSSH_SEQ_NO': 1418890732329.0, 'INDUTY_DESC': '경양식

**7835**건을 읽었고, 마지막 줄 `8-18 00:00:00.0', 'GU_NM': '동작구'}]`가 출력된 것을 볼 수 있다.

In [164]:
!type ds_open_IndividualServiceChargeService_xml.txt

[{'BSSH_NM': '삼지', 'BSSH_SEQ_NO': 1418890661033.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-763-8264', 'AR': '', 'LEGALDONG_NM': '재동', 'ADRES_CN2': '서울특별시 종로구 북촌로2길 5-7 (재동)', 'PRDLST_CODE_SE': '1418622761384', 'PRDLST_DESC': '김치찌개 백반', 'PC': 7000.0, 'CHCK_MD': '2021-08-18 00:00:00.0', 'GU_NM': '종로구'}, {'BSSH_NM': '삼지', 'BSSH_SEQ_NO': 1418890661033.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-763-8264', 'AR': '', 'LEGALDONG_NM': '재동', 'ADRES_CN2': '서울특별시 종로구 북촌로2길 5-7 (재동)', 'PRDLST_CODE_SE': '1418622761373', 'PRDLST_DESC': '삼겹살', 'PC': 14000.0, 'CHCK_MD': '2021-08-18 00:00:00.0', 'GU_NM': '종로구'}, {'BSSH_NM': '삼지', 'BSSH_SEQ_NO': 1418890661033.0, 'INDUTY_DESC': '한식', 'TLPHON_NO_CN': '02-763-8264', 'AR': '', 'LEGALDONG_NM': '재동', 'ADRES_CN2': '서울특별시 종로구 북촌로2길 5-7 (재동)', 'PRDLST_CODE_SE': '1418622761372', 'PRDLST_DESC': '된장찌개 백반', 'PC': 7000.0, 'CHCK_MD': '2021-08-18 00:00:00.0', 'GU_NM': '종로구'}, {'BSSH_NM': '김밥짱', 'BSSH_SEQ_NO': 1418890732329.0, 'INDUTY_DESC': '경양식', 'TLPHON_NO_CN': '02-

In [165]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: C826-817A

 C:\Users\zaqxs\Code\201910787\s-master\src 디렉터리

2021-09-18  오후 08:02    <DIR>          .
2021-09-18  오후 08:02    <DIR>          ..
2021-09-11  오후 03:13    <DIR>          .ipynb_checkpoints
2021-09-12  오후 08:01               649 ds_open_cardBusTime.py
2021-09-11  오전 12:35               999 ds_open_foreigners_xml.py
2021-09-15  오후 02:14               967 ds_open_gokr_ex.py
2021-09-11  오전 12:34               163 ds_open_hello.xml
2021-09-18  오후 08:02             1,448 ds_open_IndividualServiceChargeService_json.py
2021-09-18  오후 08:07            33,649 ds_open_IndividualServiceChargeService_xml.txt
2021-09-15  오후 02:22               769 ds_open_library.py
2021-09-12  오후 02:56            12,198 ds_open_library.xml
2021-09-15  오후 02:19               181 ds_open_library_exl.txt
2021-09-15  오후 02:22            11,191 ds_open_library_xml.txt
2021-09-11  오후 03:29             1,798 ds_open_subwayTime.py
2021-09-11  오전 12:34             1,755 ds_ope

`2021-09-18  오후 08:07  33,649 ds_open_IndividualServiceChargeService_xml.txt`
에서 볼 수 있듯이 **33,649**바이트가 할당되었다.

### 문제2 답

In [12]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, Java HotSpot(TM) 64-Bit Server VM, 11.0.12
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [13]:
!java -version 

java version "11.0.12" 2021-07-20 LTS
Java(TM) SE Runtime Environment 18.9 (build 11.0.12+8-LTS-237)
Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.12+8-LTS-237, mixed mode)


In [14]:
!javac -version

javac 11.0.12


In [15]:
!python --version

Python 3.7.3


In [16]:
!python3 --version

Python 3.9.7


In [17]:
import sys
print(sys.version)

3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]


In [18]:
!where python

C:\Users\zaqxs\Anaconda2\envs\py35\python.exe
C:\Users\zaqxs\Anaconda2\python.exe
C:\Users\zaqxs\AppData\Local\Programs\Python\Python36\python.exe
C:\Users\zaqxs\AppData\Local\Microsoft\WindowsApps\python.exe


In [53]:
spark.version

'3.1.2'

In [11]:
import sys
print (sys.executable)

C:\Users\zaqxs\Anaconda2\envs\py35\python.exe


### SparkSession 생성

In [12]:
import os

home=os.path.expanduser("~")
os.environ["PYSPARK_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.9-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [13]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 문제3 답

In [25]:
popRdd = spark.sparkContext\
    .textFile(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"), use_unicode=True)

In [28]:
popRdd.take(5)

['행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자',
 '의정부1동,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,의정부시,민원여권과,031-828-2466,2021-09-10',
 '의정부2동,31380,15608,15772,6.77,3.37,3.4,98.96,16410,1.91,의정부시,민원여권과,031-828-2466,2021-09-10',
 '호원1동,36124,17595,18529,7.8,3.8,4,94.96,15653,2.31,의정부시,민원여권과,031-828-2466,2021-09-10',
 '호원2동,34957,16923,18034,7.54,3.65,3.89,93.84,13683,2.55,의정부시,민원여권과,031-828-2466,2021-09-10']

In [29]:
for i in popRdd.take(5):
    print(i)

행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자
의정부1동,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,의정부시,민원여권과,031-828-2466,2021-09-10
의정부2동,31380,15608,15772,6.77,3.37,3.4,98.96,16410,1.91,의정부시,민원여권과,031-828-2466,2021-09-10
호원1동,36124,17595,18529,7.8,3.8,4,94.96,15653,2.31,의정부시,민원여권과,031-828-2466,2021-09-10
호원2동,34957,16923,18034,7.54,3.65,3.89,93.84,13683,2.55,의정부시,민원여권과,031-828-2466,2021-09-10


In [30]:
agedRdd = spark.sparkContext\
    .textFile(os.path.join("data","제주특별자치도 서귀포시_고령화비율및노령화지수현황_20210831.csv"), use_unicode=True)

In [31]:
agedRdd.take(5)

['연도별,서귀포시 인구수,65세이상 인구수 ,14세이하 인구수,고령화비율,노령화지수,데이터기준일자',
 '2012,154057,25826,22861,16.76,112.97,2021-08-31',
 '2013,155641,26936,22393,17.31,120.29,2021-08-31',
 '2014,158512,27877,22058,17.59,126.38,2021-08-31',
 '2015,164519,28979,22362,17.61,129.59,2021-08-31']

In [32]:
for i in agedRdd.take(5):
    print(i)

연도별,서귀포시 인구수,65세이상 인구수 ,14세이하 인구수,고령화비율,노령화지수,데이터기준일자
2012,154057,25826,22861,16.76,112.97,2021-08-31
2013,155641,26936,22393,17.31,120.29,2021-08-31
2014,158512,27877,22058,17.59,126.38,2021-08-31
2015,164519,28979,22362,17.61,129.59,2021-08-31


다운로드 받은 파일의 한글이 깨져 있기 때문에 csv파일을 ``utf-8``로 변환하여 사용하였다.